# 1. [threading.Thread]启动与停止线程

`threading` 库可以在单独的线程中执行任何的在 `Python` 中可以调用的对象。你可以创建一个 `Thread` 对象并将你要执行的对象以 `target` 参数的形式提供给该对象

In [1]:
# Code to execute in an independent thread
import time
def countdown(n):
    while n > 0:
        print('T-minus', n)
        n -= 1
        time.sleep(5)

# Create and launch a thread
from threading import Thread
t = Thread(target=countdown, args=(10,))
t.start()

T-minus 10


当你创建好一个线程对象后，该对象并不会立即执行，除非你调用它的 `start()` 方法（当你调用 `start()` 方法时，它会调用你传递进来的函数，并把你传递进来的参数传递给该函数）。

`Python` 中的线程会在一个单独的系统级线程中执行（比如说一个 `POSIX` 线程或者一个 `Windows` 线程），这些线程将由操作系统来全权管理。

线程一旦启动，将独立执行直到目标函数返回。你可以查询一个线程对象的状态，看它是否还在执行：

```py
if t.is_alive():
    print('Still running')
else:
    print('Completed')

```

你也可以将一个线程加入到当前线程，并等待它终止：

```py
t.join()

```


后台线程无法等待，不过，这些线程会在主线程终止时自动销毁。 

除了如上所示的两个操作，并没有太多可以对线程做的事情。

你无法结束一个线程，无法给它发送信号，无法调整它的调度，也无法执行其他高级操作。

如果需要这些特性，你需要自己添加。

比如说，如果你需要终止线程，那么这个线程必须通过编程在某个特定点轮询来退出。

你可以像下边这样把线程放入一个类中：

In [12]:
%%file ThreadTest.py
import time
class CountdownTask:
    def __init__(self):
        self._running = True

    def terminate(self):
        self._running = False

    def run(self, n):
        while self._running and n > 0:
            print('T-minus', n)
            n -= 1
            time.sleep(5)

c = CountdownTask()

from threading import Thread
t = Thread(target=c.run, args=(10,))
t.start()
c.terminate() # Signal termination 信號終止
t.join()      # Wait for actual termination (if needed) 實際停止

Writing ThreadTest.py


In [13]:
# ============ 開新 CONSOLE ============
# ------------ Run the server ------------

import os
import subprocess
# os.path.abspath {本黨位置}: D:\Google 雲端硬碟\learn\線程調用\TestOS.py
# os.path.dirname {目錄} : D:\Google 雲端硬碟\learn\線程調用
BASE_DIR = os.path.dirname(os.path.abspath('__file__'))
# 透過 cmd 呼叫
DIR = os.path.join(BASE_DIR, 'ThreadTest.py')
cmd = "python " + f'"{DIR}"'
print(cmd,BASE_DIR,sep='\n')
#  CONSOLE混雜
#os.system(cmd)
#subprocess.call(cmd)

#  NEW 一個 CONSOLE
subprocess.Popen(cmd, creationflags=subprocess.CREATE_NEW_CONSOLE)

python "D:\CODE\GitHub\py\資料結構\py3-cookbook\并发编程\ThreadTest.py"
D:\CODE\GitHub\py\資料結構\py3-cookbook\并发编程


如果线程执行一些像 `I/O` 这样的阻塞操作，那么通过轮询来终止线程将使得线程之间的协调变得非常棘手。

比如，如果一个线程一直阻塞在一个 `I/O` 操作上，它就永远无法返回，也就无法检查自己是否已经被结束了。

要正确处理这些问题，你需要利用超时循环来小心操作线程。 例子如下：

```py
class IOTask:
    def terminate(self):
        self._running = False

    def run(self, sock):
        # sock is a socket
        sock.settimeout(5)        # Set timeout period
        while self._running:
            # Perform a blocking I/O operation w/ timeout
            try:
                data = sock.recv(8192)
                break
            except socket.timeout:
                continue
            # Continued processing
            ...
        # Terminated
        return
```



## 讨论

由于全局解释锁 `（GIL）` 的原因， `Python` 的线程被限制到同一时刻只允许一个线程执行这样一个执行模型。

所以， `Python` 的线程更适用于处理 `I/O` 和其他需要并发执行的阻塞操作（比如等待 `I/O` 、等待从数据库获取数据等等），而**不是需要多处理器并行的计算密集型任务**

你可以通过 `multiprocessing` 模块在一个单独的进程中执行你的代码：

In [18]:
import multiprocessing

class CountdownTask:
    def __init__(self):
        self._running = True

    def terminate(self):
        self._running = False

    def run(self, n):
        while self._running and n > 0:
            print('T-minus', n)
            n -= 1
            time.sleep(1)

c = CountdownTask()
p = multiprocessing.Process(
    target=c.run(n=5)
)
p.start()

T-minus 5
T-minus 4
T-minus 3
T-minus 2
T-minus 1


# 2.判断线程是否已经启动